In [1]:
link = "https://api.steinhq.com/v1/storages/5e7ca978b88d3d04ae0815f6/KAQuarantineMarkers"

In [2]:
import urllib.request, json
import re


def get_data(link: str):
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read().decode())
    return data


data = get_data(link)

In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

/Users/meghanabhange/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
all_data = pd.DataFrame(data)

In [5]:
df = pd.DataFrame()
df["port-of-origin-of-journey"] = all_data["port-of-origin-of-journey"]

In [6]:
len(df["port-of-origin-of-journey"].unique())

2068

In [7]:
df["port-of-origin-of-journey"] = df["port-of-origin-of-journey"].progress_apply(
    lambda x: re.sub(r"[^\w\s]", " ", x) if x else None
)

100%|██████████| 9369/9369 [00:00<00:00, 150675.36it/s]


In [8]:
df["port-of-origin-of-journey"] = df["port-of-origin-of-journey"].progress_apply(
    lambda x: x.title() if x else None
)
len(df["port-of-origin-of-journey"].unique())

100%|██████████| 9369/9369 [00:00<00:00, 281799.91it/s]


1658

In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [12]:
def is_place(text):
    if text:
        doc = nlp(text)
        if not doc.ents:
            return False
        return doc.ents[0].label_ =="GPE"

In [13]:
def extract_place(places):
    extracted = set()
    for ele in tqdm(places):
        for text in ele.split(): 
            if is_place(text):
                extracted.add(text)
    return extracted

In [14]:
text = list(df["port-of-origin-of-journey"].dropna().unique())

In [15]:
extracted = extract_place(text)

100%|██████████| 1657/1657 [00:24<00:00, 66.82it/s]


In [16]:
import wikipedia
cities = []
countries = []
other_places = []

for text in tqdm(extracted):
    try:
        summary = str(wikipedia.summary(text))
        if ('city' in summary):
            cities.append(text)
        elif ('country' in summary):
            countries.append(text)
        else:
            other_places.append(text)
    except:
        print(f"error at {text}")

  8%|▊         | 17/204 [00:38<07:04,  2.27s/it]/Users/meghanabhange/.pyenv/versions/3.7.3/lib/python3.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/meghanabhange/.pyenv/versions/3.7.3/lib/python3.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
  9%|▉         | 18/204 [00:40<07:19,  2.36s/it]

error at Bostan


 11%|█         | 22/204 [00:49<06:57,  2.29s/it]

error at Johnessburg


 15%|█▌        | 31/204 [01:11<07:36,  2.64s/it]

...

 95%|█████████▌| 194/204 [07:39<00:24,  2.46s/it]

error at Washington


 99%|█████████▊| 201/204 [07:55<00:07,  2.48s/it]

error at Baston


100%|██████████| 204/204 [08:02<00:00,  2.37s/it]


In [17]:
len(extracted)

204

In [121]:
prefered_labels = cities + countries
prefered_labels = set(prefered_labels)
for ele in (
    "New York",
    "Abu Dhabhi",
    "Kuala Lumpur",
    "Kenya",
    "Washington",
    "Qatar",
    "Georgia",
    "Phoenix",
    "Prague",
    "Ireland",
    "Muscat",
    "Mauritius",
    "Melbourne",
    "Bahrain",
    "Oman",
    "UK",
    "Glasgow",
    "Netherlands"
):
    prefered_labels.add(ele)
prefered_labels.remove("Abhdhabi")
prefered_labels.remove("Neetherland")
prefered_labels.remove("Abu Dhabhi")

In [122]:
len(prefered_labels)

188

In [123]:
from fuzzywuzzy import process

In [134]:
def correct_text(text):
    if not text:
        return
    output = {}
    ext = process.extractOne(text, prefered_labels)
    output[ext[0]] = ext[1]
    for ele in text.split():
        ext = process.extractOne(ele, prefered_labels)
        output[ext[0]] = ext[1]
    new_text = []
#     print(output)
    for key, value in output.items():
        if value==100:
            new_text.append(key)
    if not new_text:
        for key, value in output.items():
            if value>90:
                new_text.append(key)
    if not new_text:
        for key, value in output.items():
            if value>80:
                new_text.append(key)
    if not new_text:
        return text
    return "/".join(set(new_text))

In [135]:
correct_text("Muscat Oman")

'Oman/Muscat'

In [136]:
for ele in df["port-of-origin-of-journey"].unique():
    converted = correct_text(ele)
    if not (ele==converted):
        if converted:
            print(f"converting {ele} to {converted}")

converting Mauritis to Mauritius
converting Dhubai to Dubai
converting Baharain to Bahrain
converting Muscut Oman to Oman
converting Jeddah to Jeedah
converting Cedar Raptds Ia Usa to Bulgaria
converting Sofia Bulgria to Bulgaria
converting Dbai to Dubai
converting Muscat  Oman to Oman/Muscat
converting Birmingham Uk to UK/Birmingham
converting London Health Row Uk to London/UK
converting Glagow Uk to UK
converting Singapur to Singapore
converting Doha  Qatar to Doha/Qatar
converting Londan to London
converting Neetherland to Netherlands
converting Sydney Australia to Australia
converting Munich  Germany to Germany
converting Frankfruit  Germany to Germany
converting Dubai Dxb to Dubai
converting Doha   Qatar  to Doha/Qatar
converting Doha   Qatar to Doha/Qatar
converting Berlini Germeny to Berlin
converting Brenen Frankfket Transit  to Bremen
converting London Healthrow to London
converting Abudhabi to Abu Dhabi
converting Breman Frankfort to Bremen/Frankfurt
converting Herat Nato  Af

converting Portland Usa to Poland
converting Johar Baharu Malaysia to Malaysia
converting Riyadh Saudi Arabia to Riyadh
converting Tulsa Ok Usa to Tokyo
converting Changi Airport Singapore to Singapore
converting Portugul to Portugal
converting Addis Ababa to Ababa
converting London Uk to London/UK
converting Dubai Uae to Dubai
converting Doha Qatar to Doha/Qatar
converting Dubai  Uae to Dubai
converting Abudhabi Uae to Abu Dhabi
converting Uae  Abu Dhabi to Abu Dhabi
converting Kl Malaysia to Malaysia
converting K L Malaysia to Malaysia
converting Edmonton  Ab Canada to Canada
converting Dubai U A E to Dubai
converting Kul Malaysia to Malaysia
converting Irrland to Ireland
converting Irelnad to Ireland
converting San Francise to France
converting Perth  Austrailia to Australia
converting Muscat Oman to Oman/Muscat
converting Muscat Int Airport to Muscat
converting Uae Dubai to Dubai
converting Muscut to Muscat
converting Jeddah Saudi Arabia to Jeedah
converting Dubhai to Dubai
convert

 31%|███       | 2869/9369 [01:30<02:39, 40.67it/s]

converting Budapest  Hu to Hungary
converting Lisbon   Portugal to Portugal
converting Ottava Canada to Canada
converting Huston Texas to Texas
converting Ohawa Canada to Canada
converting Cardiff   Uk to UK
converting London  Uk to London/UK
converting Kualalampur to Kulalampur
converting Phoenilx Usa to Phoenix
converting K T M to UK/Milan/Hartford
converting London  Uk  to London/UK
converting Japan Osaka
 Kansai to Japan
converting Montreal Canada to Canada
converting Jfk  New York to New York
converting Dar Ls Salaam to Tulsa
converting Lome  Togo to Lome
converting New York Jfk to New York
converting Moputo  Mozanbique to Mozanbique
converting New Delhi to Delhi
converting New Zealand to New York
converting Kansas City to Kansas
converting Addis Ababa Ethiopia  to Ababa/Ethiopia
converting Etiopia to Ethiopia
converting Add Addis Ababa to Ababa
converting Phnom Penh to Phnom
converting Landon  Uk to UK
converting Chicagoa to Chicago
converting Cork  Ireland to Ireland
converting 

converting Philadesphia Usa to Philadelphia
converting Londan Uk to UK
converting Dublin Irland to Dublin
converting Accra  Ghana  to Ghana
converting Ca Usa Dubai to Dubai
converting Atlanta Georgia to Atlanta/Georgia
converting Denver Co Usa  to Denver
converting Montreal  Canada to Canada
converting Arl Doha Banaglore to Doha
converting Arl Doha to Doha
converting Dallas Dfw to Dallas
converting Dubai Uac to Dubai
converting Helsinki Finland to Finland
converting Ireland Cork to Ireland
converting Taipei  Taiwan to Taipei/Taiwan
converting Heathrow  London to London
converting Aukland New Zeland to Auckland
converting Newark Dubai to Dubai
converting Athenes to Athens
converting Hngary to Hungary
converting Helsink Finland  to Finland
converting Go Thenburg Sweden to Sweden
converting Abu to Abu Dhabi
converting Nnether Lands to Netherlands
converting London Heathrow  Uk to London/UK
converting St Louis  Usa to Belfast/Louisiana
converting Krakow Poland to Poland
converting London  

converting Buenos Aires Argentina to Argentina
converting Cincinati  Oh to Cincinnati
converting Atalnta  Usa to Atlanta
converting Helsenki Fin Land  to Netherlands/Finland
converting Torowto Kannada to Toronto
converting South Sudan to Sudan
converting Taipei Taiwan to Taipei/Taiwan
converting Swizerland to Switzerland
converting Sieani Reap Cambodia to Cambodia
converting Usa  Philadelphia to Philadelphia
converting Kuwati Dubai to Dubai
converting Brusels to Brussels
converting Frank Frut to Frankfurt
converting London  Heattrow to London
converting Austrlia to Australia
converting Dallas Texas to Dallas/Texas
converting Penang Malaysia to Malaysia
converting Bostan  Usa to Boston
converting Boston Ma Usa to Boston
converting Usa Germany to Germany
converting Dubai T3 to Dubai
converting Allanta to Atlanta
converting Dubai International Airport to Dubai
converting Gdansk  Poland to Poland
converting France Furt Airport to France
converting London   Heathrow  Terminal 2  to London
c

converting London   Heathrow to London
converting Washngton Dc Usa to Washington
converting Sweden Stockholm  to Sweden
converting Dubai Uee to Dubai
converting Aruba Boston to Aruba/Boston
converting Uk  to UK
converting London Heathtow to London
converting Warsaw  Poland to Poland/Warsaw
converting Porto Portugal to Portugal
converting Zurich Dubai to Zurich/Dubai
converting France Lyon to France
converting Atlantica Usa to Atlanta
converting Glasgow  Uk to Glasgow/UK
converting Toronto  Canada  to Canada/Toronto
converting Milan
 Via Dubai  to Dubai/Milan
converting Kaulalalumpur to Kuala Lumpur
converting Atlantica to Atlanta
converting Addisa Ababa to Ababa
converting K  Lumpur to Kuala Lumpur/UK
converting Plaisance Mauritius to Mauritius
converting Siemreap Cambodia to Cambodia
converting Uk  London Heathrow  to London/UK
converting Uk Moncgaster to UK
converting Paris Muscat to Paris/Muscat
converting Dusseldorf Germany to Germany
converting Heathiru G R to Hartford/Bulgaria
co

In [137]:
df["new-port-of-origin-of-journey"] = df["port-of-origin-of-journey"].progress_apply(
    correct_text
)


  3%|▎         | 255/9369 [00:06<03:55, 38.66it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

  4%|▍         | 360/9369 [00:09<03:57, 37.91it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

  6%|▌         | 525/9369 [00:13<03:38, 40.46it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

  7%|▋         | 680/9369 [00:17<04:05, 35.46it/s]


 13%|█▎        | 1248/9369 [00:31<02:52, 46.95it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 14%|█▍        | 1335/9369 [00:33<03:06, 43.04it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 15%|█▍        | 1401/9369 [00:34<02:56, 45.23it/s]


 18%|█▊        | 1672/9369 [00:40<02:59, 42.89it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 20%|█▉        | 1843/9369 [00:45<03:41, 33.93it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 22%|██▏       | 2079/9369 [00:50<02:42, 44.77it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 23%|██▎       | 2123/9369 [00:51<03:44, 32.32it/s]


 30%|███       | 2819/9369 [01:10<02:39, 41.14it/s]


 30%|███       | 2839/9369 [01:10<03:14, 33.54it/s]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']

 37%|███▋      | 3496/9369 [01:28<02:14, 43.54it/s]


...

 77%|███████▋  | 7184/9369 [02:55<00:48, 44.95it/s]


 85%|████████▍ | 7959/9369 [03:14<00:33, 41.71it/s]


 93%|█████████▎| 8707/9369 [03:32<00:17, 37.58it/s]


100%|█████████▉| 9325/9369 [03:52<00:01, 41.05it/s]


100%|██████████| 9369/9369 [03:53<00:00, 40.04it/s]


In [138]:
df.to_csv("testing3.csv")

In [139]:
len(df["new-port-of-origin-of-journey"].unique())

821

In [145]:
len(pd.read_csv("testing.csv")["new-port-of-origin-of-journey"].unique())

1002